In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
data = pd.read_csv("diabetes.csv")

In [3]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
data.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [5]:
data.iloc[:,1:-1].replace(0,np.nan,inplace=True)

In [6]:
data['SkinThickness'] = data['SkinThickness'].replace(99,np.nan)

In [7]:
data["Pregnancies"] = data["Pregnancies"].replace(np.nan,0)

In [8]:
data['Glucose'].fillna(data['Glucose'].mean(), inplace = True)
data['BloodPressure'].fillna(data['BloodPressure'].mean(), inplace = True)
data['SkinThickness'].fillna(data['SkinThickness'].median(), inplace = True)
data['Insulin'].fillna(data['Insulin'].median(), inplace = True)
data['BMI'].fillna(data['BMI'].median(), inplace = True)

In [9]:
from sklearn.utils import resample
data["Outcome"].value_counts()
df_min = data[data['Outcome']==1] 
df_max = data[data['Outcome']==0]

df_min_upscaled = resample(df_min, replace=True,
                           n_samples=500,
                           random_state=0) 
data = pd.concat([df_max,df_min_upscaled]) 
data.Outcome.value_counts()

1    500
0    500
Name: Outcome, dtype: int64

In [10]:
X= data.iloc[:,:-1]

In [11]:
y= data.iloc[:,-1]

In [12]:
X.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train= sc_X.fit_transform(X_train)
X_test= sc_X.transform(X_test)

In [15]:
import xgboost as xgb
model=xgb.XGBClassifier(learning_rate= 0.48264028704212025, max_depth=11,n_estimators=151,gamma=0.1,colsample_bytree=0.7,min_child_weight=3)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

In [16]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test,y_pred)

In [17]:
score

0.92

In [18]:
from pickle import dump
dump(model, open('diabetes_model.sav', 'wb'))
dump(sc_X, open('diabetes_scaler.sav', 'wb'))